In [1]:
import pandas as pd
from sklearn.svm import SVR
import numpy as np

In [2]:
pm_10 = pd.read_csv('./opendata_pm10_weather.csv')
no2 = pd.read_csv('./opendata_no2_weather.csv')

In [4]:
x = pm_10.loc[:, pm_10.columns != 'value_tomorrow']
x = x.drop(labels=["type", "date_format", 'Unnamed: 0', 'temp_min', 'temp_max', 'location'], axis=1)


In [5]:
x = x.fillna(0)
x = x[:-1]

y = pm_10["value_tomorrow"]
y = y[:-1]
# x = x.values.tolist()


In [6]:
print(x.describe())
print(y.describe())


          pressure     humidity   wind_speed     rain_24h      snow_3h  \
count  5088.000000  5088.000000  5088.000000  5088.000000  5088.000000   
mean   1009.772883    70.634616     2.733980     0.003295     0.000087   
std      21.008419    14.819337     1.419910     0.074459     0.002524   
min     903.500000     0.000000     0.000000     0.000000     0.000000   
25%    1009.263158    60.203125     1.782609     0.000000     0.000000   
50%    1015.000000    71.950000     2.434783     0.000000     0.000000   
75%    1019.600000    82.666667     3.388889     0.000000     0.000000   
max    1038.416667    99.363636    11.210526     2.032000     0.073529   

        clouds_all    temp_cent  temp_max_cent  temp_min_cent        value  
count  5088.000000  5088.000000    5088.000000    5088.000000  5088.000000  
mean     34.355412    12.963655      13.402431      11.705862    39.545160  
std      29.308187     8.415748       8.188454       8.959687    43.344769  
min       0.000000    -9.

In [ ]:
# y = np.array(y).astype(np.float64)
# X = np.array(x).astype(np.float64)

In [56]:
seasons = {'winter': 0, 'spring': 1, 'summer': 2, 'autumn': 3}
X = []
for x_ in x.values.tolist():
    new_x = x_
    new_x[10] = seasons[x_[10]]
    X.append(new_x)
    
    
from sklearn.preprocessing import MinMaxScaler
X = MinMaxScaler().fit_transform(X)

In [39]:
x_train = X[:4500]
y_train = y[:4500]

x_test = X[4501:]
y_test = y[4501:]

In [73]:
regressor = SVR(kernel='rbf', C=10, gamma=0.001)
regressor.fit(x_train, y_train)
predicted = regressor.predict(x_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error

mean_absolute_error(predicted, y_test)
mean_squared_error(predicted, y_test)

917.64535737868414

In [74]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adam


In [81]:
model = Sequential()
model.add(Dense(50, activation='linear', input_shape=(11,)))
model.add(Dropout(0.2))
model.add(Dense(40, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(30, activation='linear'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))

In [82]:
model.compile(loss='mean_absolute_error',
              optimizer=Adam())

history = model.fit(x_train, y_train,
                    batch_size=16,
                    epochs=100,
                    verbose=1,
                    validation_split=0.2)

Train on 3600 samples, validate on 900 samples
Epoch 1/100
3600/3600 [==============================] - 2s - loss: 36.2036 - val_loss: 9.8143

In [85]:
predicted = model.predict(x_test)

In [86]:
mean_absolute_error(predicted, y_test)

20.88579269548088